BLOCCO 1





In [ ]:
from qiskit_aer.noise import QuantumError, NoiseModel

# 1. Input: numero di qubit e probabilità di rumore
n = 2
p = 0.2

# 2. Genera l'osservabile H
h = 9 / n
d = 2 ** n  # dimensione dello spazio degli stati
Z = np.array([[1, 0], [0, -1]])
I = np.eye(2)

H_mat = np.zeros((d, d))
for k in range(0, n - 1, 2):
    op = 1
    for qubit in range(n):
        if qubit == k or qubit == k + 1:
            op = np.kron(op, Z)
        else:
            op = np.kron(op, I)
    H_mat += op
H_mat = h * H_mat
print("Osservabile H generato:\n", H_mat)

# 3. Creazione del circuito ansatz semplice
theta = 0.3
qc = QuantumCircuit(n)
for i in range(n):
    qc.rx(theta, i)
for i in range(n - 1):
    qc.cx(i, i + 1)

# 4. Definizione del canale di rumore personalizzato
GHZ = np.zeros((d, 1), dtype=complex)
GHZ[0] = 1 / np.sqrt(2)
GHZ[-1] = 1 / np.sqrt(2)

K1 = np.sqrt(1 - p) * np.eye(d)
K2_list = []
for i in range(d):
    ket_i = np.zeros((d, 1), dtype=complex)
    ket_i[i, 0] = 1.0
    K2 = np.sqrt(p / d) * np.dot(GHZ, ket_i.conj().T)
    K2_list.append(K2)
kraus_ops = [K1] + K2_list

noise_error = QuantumError(kraus_ops)
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(noise_error, ['id'])

for i in range(n):
    qc.id(i)

print("\nCircuito ansatz con rumore:")
print(qc.draw())

# 5. Simulazione del circuito con il noise model
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, noise_model=noise_model, shots=1024, parameter_binds=[{theta: 0.5}])
result = job.result()
counts = result.get_counts(qc)
print("\nRisultati della simulazione (counts):", counts)

# (Opzionale) Simulazione dello statevector finale con il noise model
backend_sv = Aer.get_backend('statevector_simulator')
job_sv = execute(qc, backend_sv, noise_model=noise_model, parameter_binds=[{theta: 0.5}])
result_sv = job_sv.result()
statevector = result_sv.get_statevector(qc)
print("\nStatevector finale (con rumore):\n", statevector)


Osservabile H generato:
 [[ 4.5  0.   0.   0. ]
 [ 0.  -4.5  0.   0. ]
 [ 0.   0.  -4.5  0. ]
 [ 0.   0.   0.   4.5]]


NoiseError: "Invalid type of input is found around '[[0.89442719 0.         0.         0.        ]\n [0.         0.89442719 0.         0.        ]\n [0.         0.         0.89442719 0.        ]\n [0.         0.         0.         0.89442719]]'"

BLOCCO 2

In [21]:
import qiskit as qc
from qiskit_aer import Aer
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import DensityMatrix, Kraus, Operator, SparsePauliOp
from IPython.display import display, Latex

# Parametri
n = 4  # Numero di qubit
shots = 10000  # Numero di esecuzioni
L_reps = 2  # Numero di ripetizioni del circuito
p = 0.1  # Probabilità di applicare l'operatore di Kraus
h = 1  # Coefficiente nell'Hamiltoniano

# Funzione per generare parametri casuali
def generate_random_parameters(n):
    return 2 * np.pi * np.random.rand(2 * n)

# Funzione per creare la catena di CNOT come circuito
def create_cnot_chain_circuit(n):
    cnot_circuit = QuantumCircuit(n)
    for i in range(n - 1):
        cnot_circuit.cx(i, i + 1)
    return cnot_circuit

# Costruzione della matrice per la catena di CNOT
cnot_circuit = create_cnot_chain_circuit(n)
cnot_operator = Operator(cnot_circuit)

# Definizione degli operatori di Kraus
I_n = np.eye(2**n)  # Identità su n qubit
K0 = np.sqrt(1 - p) * I_n
K1 = np.sqrt(p) * cnot_operator.data  # Matrice della catena di CNOT
kraus_channel = Kraus([K0, K1])  # Creazione del canale di Kraus

# Creazione di W (Hamiltoniano)
pauli_strings = []
coefficients = []

for k in range(n):
    coeff = h * 2**(n / 2)  # Coefficiente del termine
    coefficients.append(coeff)

    # Stringa di Pauli per Z_k ⊗ Z_{k+1}
    pauli_string = ['I'] * n
    pauli_string[k] = 'Z'  # Z_k
    pauli_string[(k + 1) % n] = 'Z'  # Z_{k+1} (periodico)
    pauli_strings.append("".join(pauli_string))

# Creazione dell'Hamiltoniano come SparsePauliOp
W = SparsePauliOp(pauli_strings, coeffs=np.array(coefficients, dtype=complex))

# Mostra l'Hamiltoniano
print("Hamiltoniano W:")
print(W)

# Creazione del circuito quantistico
qc = QuantumCircuit(n, n)

# Ripeti il circuito L volte
for _ in range(L_reps):
    # Genera parametri casuali ad ogni iterazione
    parameters = generate_random_parameters(n)

    # Applica rotazioni
    for i in range(n):
        qc.rx(parameters[2 * i], i)
        qc.rz(parameters[2 * i + 1], i)

    # Applica il canale di Kraus
    qc.append(kraus_channel, list(range(n)))

    # Applica le operazioni CNOT personalizzate
    qc.cx(0, 1)
    qc.cx(3, 2)
    qc.cx(0, 2)
    qc.cx(2, 0)
    qc.cx(0, 2)
    qc.cx(1, 3)
    qc.cx(3, 1)
    qc.cx(1, 3)

# Creazione della matrice densità finale
rho = DensityMatrix.from_instruction(qc)

# Visualizzare la matrice densità finale
latex_rho = r"$\rho = \left(\begin{matrix}" + \
    f"{rho.data[0, 0]:.2f} & {rho.data[0, 1]:.2f} & \cdots & {rho.data[0, -1]:.2f} \\" + \
    f"{rho.data[1, 0]:.2f} & {rho.data[1, 1]:.2f} & \cdots & {rho.data[1, -1]:.2f} \\" + \
    r"\vdots & \vdots & \ddots & \vdots \\" + \
    f"{rho.data[-1, 0]:.2f} & {rho.data[-1, 1]:.2f} & \cdots & {rho.data[-1, -1]:.2f}" + \
    r"\end{matrix}\right)$"
display(Latex(latex_rho))

# Misura dei qubit
qc.measure(range(n), range(n))

# Simulazione
simulator = Aer.get_backend('aer_simulator')
qc = transpile(qc, simulator)
result = simulator.run(qc, shots=shots).result()

# Calcolo della traccia L = Tr(W * rho)
# Convertiamo W in una matrice densa e calcoliamo il prodotto con rho
W_matrix = W.to_matrix()
L = np.trace(np.dot(W_matrix, rho.data))

# Stampa del risultato
print(f"Il valore della traccia L è: {L:.5f}")

# Mostra il circuito
print("\nCircuito con la misura e il punto di calcolo di rho:")
print(qc.draw(output="text"))

Hamiltoniano W:
SparsePauliOp(['ZZII', 'IZZI', 'IIZZ', 'ZIIZ'],
              coeffs=[4.+0.j, 4.+0.j, 4.+0.j, 4.+0.j])


<IPython.core.display.Latex object>

Il valore della traccia L è: 0.25387+0.00000j

Circuito con la misura e il punto di calcolo di rho:
global phase: 5.3469
     ┌──────────────────────────┐┌────────┐               ┌───┐          »
q_0: ┤ U3(0.80152,-1.1558,-π/2) ├┤0       ├──■────■───────┤ X ├───────■──»
     ├─────────────────────────┬┘│        │┌─┴─┐  │       └─┬─┘┌───┐  │  »
q_1: ┤ U3(1.5075,-1.0359,-π/2) ├─┤1       ├┤ X ├──┼────■────┼──┤ X ├──┼──»
     ├─────────────────────────┤ │  kraus │├───┤┌─┴─┐  │    │  └─┬─┘┌─┴─┐»
q_2: ┤ U3(2.0729,-0.43575,π/2) ├─┤2       ├┤ X ├┤ X ├──┼────■────┼──┤ X ├»
     └┬────────────────────────┤ │        │└─┬─┘└───┘┌─┴─┐       │  └───┘»
q_3: ─┤ U3(1.8166,2.8978,-π/2) ├─┤3       ├──■───────┤ X ├───────■───────»
      └────────────────────────┘ └────────┘          └───┘               »
c: 4/════════════════════════════════════════════════════════════════════»
                                                                         »
«     ┌───────────────────────┐                       